In [1]:
import pandas as pd
import numpy as np
import requests
import json
import prettytable
from tabulate import tabulate
from datetime import date, time, datetime

In [2]:
bls_series = {"NYC": 
                   {"seriesID": ["LAUCN360610000000003", "LAUCN360470000000003", "LAUCN360050000000003", "LAUCN360850000000003", "LAUCN360810000000003"], 
                    "counties": ["New York", "Kings", "Bronx", "Richmond", "Queens"]}, 
               "LosAngeles": 
                   {"seriesID": ["LAUCN060370000000003"], 
                    "counties": ["Los Angeles"]}, 
               "Chicago": 
                   {"seriesID": ["LAUCN170310000000003"], 
                    "counties": ["Cook"]},
               "Houston": 
                   {"seriesID": ["LAUCN482010000000003"], 
                    "counties": ["Harris"]},
               "Phoenix": 
                   {"seriesID": ["LAUCN040130000000003"], 
                    "counties": ["Maricopa"]},
              }

In [3]:
def get_uem(series_dict):
    df_data = pd.DataFrame()
    for key in series_dict.keys():    
        headers = {'Content-type': 'application/json'}
        data = json.dumps({"seriesid": series_dict[key]['seriesID'],"startyear":"2019", "endyear":"2020", "registrationkey": "c4f77fe780da448b8591b1c9bc154f26"})
        p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
        json_data = json.loads(p.text)

        for series, county in zip(json_data['Results']['series'], series_dict[key]['counties']):
            df = pd.DataFrame(series['data']).drop(columns={'footnotes', 'periodName'})
            df['period'] = (df['year']+df['period']).replace({'M':'-'}, regex=True)
            df['period'] = pd.to_datetime(df['period']).dt.strftime('%Y-%m')
            df = df.drop('year', axis=1).iloc[::-1].rename(columns={'value': county})
            if df_data.empty: df_data = pd.concat([df_data, df])
            else: df_data = df_data.merge(df, on='period')
    return df_data

In [4]:
df_uem = get_uem(bls_series)
print(tabulate(df_uem, headers='keys', tablefmt='psql'))

+----+----------+------------+---------+---------+------------+----------+---------------+--------+----------+------------+
|    | period   |   New York |   Kings |   Bronx |   Richmond |   Queens |   Los Angeles |   Cook |   Harris |   Maricopa |
|----+----------+------------+---------+---------+------------+----------+---------------+--------+----------+------------|
|  0 | 2019-01  |        4.3 |     4.9 |     6.5 |        4.7 |      4.2 |           5   |    5   |      4.5 |        4.7 |
|  1 | 2019-02  |        3.9 |     4.6 |     6   |        4.3 |      3.9 |           4.6 |    4.5 |      4.1 |        4.3 |
|  2 | 2019-03  |        3.6 |     4.2 |     5.5 |        4   |      3.6 |           4.5 |    4.3 |      3.8 |        4.3 |
|  3 | 2019-04  |        3.2 |     3.7 |     4.9 |        3.5 |      3.2 |           4.1 |    4   |      3.4 |        4   |
|  4 | 2019-05  |        3.4 |     3.9 |     5   |        3.6 |      3.3 |           4.1 |    3.7 |      3.4 |        4.1 |
|  5 | 2

In [5]:
df_uem.to_csv('../datasets/unemployment-2019-2021.csv')